In [ ]:
%pip install pandas
%pip install numpy 
%pip install matplotlib
%pip install seaborn

In [3]:
import pandas as pd
import numpy as np  
import matplotlib.pyplot as plt
import seaborn as sns
import re as re
from collections import Counter

In [4]:
# Carregar cada CSV limpo
df_netflix = pd.read_csv('C:/Users/nise_/OneDrive/Documentos/Digital College/Python/DA_18/streaming/Dados_tratados/data_netflix_clean.csv')
df_hbo = pd.read_csv('C:/Users/nise_/OneDrive/Documentos/Digital College/Python/DA_18/streaming/Dados_tratados/data_hbo_clean.csv')
df_apple = pd.read_csv('C:/Users/nise_/OneDrive/Documentos/Digital College/Python/DA_18/streaming/Dados_tratados/df_apple_clean.csv')
df_prime = pd.read_csv('C:/Users/nise_/OneDrive/Documentos/Digital College/Python/DA_18/streaming/Dados_tratados/data_prime_video_clean.csv')


print("📂 DATASETS CARREGADOS:")
print(f"Netflix: {len(df_netflix):,} registros")
print(f"HBO Max: {len(df_hbo):,} registros") 
print(f"Apple TV: {len(df_apple):,} registros")
print(f"Prime Video: {len(df_prime):,} registros")

📂 DATASETS CARREGADOS:
Netflix: 21,391 registros
HBO Max: 9,776 registros
Apple TV: 18,692 registros
Prime Video: 72,943 registros


In [5]:
# Adicionar coluna de plataforma
df_netflix['platform'] = 'Netflix'
df_hbo['platform'] = 'HBO Max'
df_apple['platform'] = 'Apple TV+'
df_prime['platform'] = 'Prime Video'

In [6]:

# ==========================================
# UNINDO OS DATASETS
# ==========================================

platforms = {
    'Netflix': df_netflix,
    'HBO Max': df_hbo, 
    'Apple TV': df_apple,
    'Prime Video': df_prime
}
# Concatenar todos os DataFrames em um único DataFrame
df_streaming = pd.concat([df_netflix, df_hbo, df_apple, df_prime], ignore_index=True)


print(f"\n🎬 DATASET UNIFICADO CRIADO:")
print(f"Total de registros: {len(df_streaming):,}")
print(f"Total de colunas: {len(df_streaming.columns)}")
print(f"Colunas: {list(df_streaming.columns)}")



🎬 DATASET UNIFICADO CRIADO:
Total de registros: 122,802
Total de colunas: 8
Colunas: ['title', 'type', 'genres', 'releaseYear', 'imdbId', 'imdbAverageRating', 'imdbNumVotes', 'platform']


In [7]:
# ==========================================
# Extrai o perfil médio por plataforma: ano, nota e top 3 gêneros
# ==========================================
def extrair_perfil_medio_por_plataforma(df):
    
    # Garantir que as colunas estão no formato correto
    df['releaseYear'] = pd.to_numeric(df['releaseYear'], errors='coerce')
    df['imdbAverageRating'] = pd.to_numeric(df['imdbAverageRating'], errors='coerce')
    
    # Explodir os gêneros para análise individual
    df_exploded = df.copy()
    df_exploded['genres_list'] = df_exploded['genres'].str.split(', ')
    df_exploded = df_exploded.explode('genres_list')
    
    # Calcular métricas por plataforma
    perfil_basico = df.groupby('platform').agg({
        'releaseYear': 'mean',
        'imdbAverageRating': 'mean'
    }).round(2)
    
    # Top 3 gêneros por plataforma
    top_generos = (df_exploded.groupby('platform')['genres_list']
                   .value_counts()
                   .groupby('platform')
                   .head(3)
                   .reset_index())
    
    # Reformatar os gêneros em string
    # Reformatar os gêneros em string e manter a coluna 'platform'
    generos_formatados = (top_generos.groupby('platform')['genres_list']
                          .apply(lambda x: ', '.join(x))
                          .reset_index())
    generos_formatados.columns = ['platform', 'Top_3_Generos']

    
    # Juntar tudo
    resultado = perfil_basico.merge(generos_formatados, on='platform')
    resultado.columns = ['Ano_Medio', 'Nota_Media_IMDb', 'Top_3_Generos']
    
    return resultado


In [8]:

def perfil(df):
    # Ano médio e nota média
    stats = df.groupby('platform').agg({
        'releaseYear': lambda x: round(x.mean(), 0),
        'imdbAverageRating': lambda x: round(x.mean(), 2)
    })
    
    # Top 3 gêneros
    generos = {}
    for plataforma in df['platform'].unique():
        dados_plat = df[df['platform'] == plataforma]
        todos_generos = []
        for genero_str in dados_plat['genres'].dropna():
            todos_generos.extend(genero_str.split(', '))
        
        top3 = pd.Series(todos_generos).value_counts().head(3).index.tolist()
        generos[plataforma] = ', '.join(top3)
    
    # Combinar resultados
    stats['Top_3_Generos'] = stats.index.map(generos)
    return stats


In [9]:
# Executar
print("\n📊 PERFIL MÉDIO POR PLATAFORMA:")
resultado = perfil(df_streaming)
print(resultado)



📊 PERFIL MÉDIO POR PLATAFORMA:
             releaseYear  imdbAverageRating               Top_3_Generos
platform                                                               
Apple TV+         2007.0               6.41      Drama, Comedy, Romance
HBO Max           2012.0               6.70  Drama, Comedy, Documentary
Netflix           2013.0               6.43       Drama, Comedy, Action
Prime Video       2008.0               6.00       Drama, Comedy, Action
